In [29]:
import os
import shutil
import argparse
from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain.prompts import ChatPromptTemplate
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.document import Document


# Front-End Web
import gradio as gr


In [30]:
DATA_PATH = 'D:\\Workspace Irede\\llamav2_local\\docs\\chroma'
CHROMA_PATH = 'D:\\Workspace Irede\\llamav2_local\\chroma'
# Caminho para o modelo Ollama3
model = Ollama(model="llama3")

In [31]:
def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

documents = load_documents()

def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

documents = load_documents()
chunks = split_documents(documents)

In [32]:
def calculate_chunk_ids(chunks):

    # This will create IDs
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page metadata
        chunk.metadata["id"] = chunk_id

    return chunks

def add_to_chroma(chunks: list[Document]):
    db = Chroma(
        persist_directory=CHROMA_PATH, embedding_function=get_embedding_function()
    )

    # Calculate Page IDs.
    chunks_with_ids = calculate_chunk_ids(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("✅ No new documents to add")

In [33]:
def get_embedding_function():
    model_name = "BAAI/bge-small-en"
    model_kwargs = {"device": "cpu"}
    encode_kwargs = {"normalize_embeddings": True}
    hf = HuggingFaceBgeEmbeddings(
        model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
    )
    return hf

# creates a command line interface for the user to perform queries
def create_CLI_main():
    parser = argparse.ArgumentParser()
    parser.add_argument("query_text", type=str, help="The query text.")
    args = parser.parse_args()
    query_text = args.query_text
    query_rag(query_text)

PROMPT_TEMPLATE = """
Você é um assistente virtual, que responde exclusivamente em português.
Use o contexto fornecido para responder à pergunta de forma clara e concisa.
Se e pergunta for fora do contexto dos PDF, responda que não pode responder fora do tópico.:

{context}

---

Answer the question based on the above context: {question}
"""

def query_rag(query_text: str, temperature: float = 0.1):
    # Prepare the DB.
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    # print(prompt)

    response_text = model.invoke(prompt, temperature=temperature)

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text

In [34]:
def clear_database():
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

# Checks if the database should be cleaned, loads the documents, breaks them into smaller chunks, and adds those chunks to the database
def main():

    # Check if the database should be cleared (using the --clear flag).
    parser = argparse.ArgumentParser()
    parser.add_argument("--reset", action="store_true", help="Reset the database.")
    args = parser.parse_args()
    if args.reset:
        print("✨ Clearing Database")
        clear_database()

    # Create (or update) the data store.
    documents = load_documents()
    chunks = split_documents(documents)
    add_to_chroma(chunks)

In [35]:
def chat(query):
    return query_rag(query)

# Create the chat interface using Gradio
iface = gr.Interface(fn=chat, inputs="text", outputs="text", title="Chat Homero APP")

if __name__ == "__main__":
    iface.launch(debug=True)
    main()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Response: Bom dia!
Sources: ['D:\\Workspace Irede\\llamav2_local\\docs\\chroma\\K8S - Definição, Comparação, Cluster de Nó Único.pdf:22:0', 'D:\\Workspace Irede\\llamav2_local\\docs\\chroma\\Tipos de Computação em Nuvem - IaaS, PaaS e SaaS.pdf:1:2', 'D:\\Workspace Irede\\llamav2_local\\docs\\chroma\\Tipos de Computação em Nuvem - IaaS, PaaS e SaaS.pdf:1:1', 'D:\\Workspace Irede\\llamav2_local\\docs\\chroma\\K8S - Definição, Comparação, Cluster de Nó Único.pdf:2:0', 'D:\\Workspace Irede\\llamav2_local\\docs\\chroma\\K8S - Definição, Comparação, Cluster de Nó Único.pdf:19:0']
Response: Kubernetes é um sistema distribuído que forma um cluster, podendo incluir máquinas físicas e virtuais, tanto on-premise quanto na nuvem.
Sources: ['D:\\Workspace Irede\\llamav2_local\\docs\\chroma\\K8S - Definição, Comparação, Cluster de Nó Único.pdf:4:0', 'D:\\Workspace Irede\\llamav2_local\\docs\\chroma\\K8S - Definição, Comparação, Cluster de Nó Único.pdf:1:0', 'D:\\Workspace Irede\\llamav2_local\\docs\

usage: ipykernel_launcher.py [-h] [--reset]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\micha\AppData\Roaming\jupyter\runtime\kernel-v2-5788PHG4ywPqyh7k.json


SystemExit: 2

c:\Users\micha\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
